<a href="https://colab.research.google.com/github/JayThibs/Weak-Supervised-Learning-Case-Study/blob/main/text_classifier/notebooks/03_dbpedia_14_snorkel_dataset_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snorkel --quiet
!pip install datasets --quiet

     |████████████████████████████████| 153kB 6.0MB/s 
     |████████████████████████████████| 3.8MB 6.3MB/s 
     |████████████████████████████████| 1.8MB 42.6MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 225kB 5.9MB/s 
     |████████████████████████████████| 112kB 14.7MB/s 
     |████████████████████████████████| 245kB 17.2MB/s 


In [33]:
import pandas as pd
from snorkel.labeling import labeling_function
from datasets import load_dataset
import re

In [3]:
dbpedia_dataset = load_dataset('dbpedia_14')

Dataset d_bpedia14 downloaded and prepared to /root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e. Subsequent calls will reuse this data.


In [4]:
dbpedia_dataset = dbpedia_dataset.rename_column("label", "labels")

In [5]:
def merge_title_with_content(example):
    example["content"] = example["title"] + " " + example["content"]
    return example

In [6]:
dbpedia_dataset = dbpedia_dataset.map(merge_title_with_content, num_proc=10)

In [21]:
dbpedia_dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 70000
    })
})

In [28]:
data = {'labels': dbpedia_dataset['train']['labels'],
        'title': dbpedia_dataset['train']['title'],
        'content': dbpedia_dataset['train']['content']}

dbpedia_train_df = pd.DataFrame(data)
dbpedia_train_df['content'] = dbpedia_train_df['content'].str.lower()
dbpedia_train_df

,labels,title,content
0,0,E. D. Abbott Ltd,e. d. abbott ltd abbott of farnham e d abbott limited was a british coachbuilding business based in farnham surrey trading under that name from 1929. a major part of their output was under sub-contract to motor vehicle manufacturers. their business closed in 1972.
1,0,Schwan-Stabilo,schwan-stabilo schwan-stabilo is a german maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. it is the world's largest manufacturer of highlighter pens stabilo boss.
2,0,Q-workshop,q-workshop q-workshop is a polish company located in poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). they also run an online retail store and maintainan active forum community.q-workshop was established in 2001 by patryk strzelewicz – a student from poznań. initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablished.
3,0,Marvell Software Solutions Israel,marvell software solutions israel marvell software solutions israel known as radlan computer communications limited before 2007 is a wholly owned subsidiary of marvell technology group that specializes in local area network (lan) technologies.
4,0,Bergan Mercy Medical Center,bergan mercy medical center bergan mercy medical center is a hospital located in omaha nebraska. it is part of the alegent health system.
...,...,...,...
559995,13,Barking in Essex,barking in essex barking in essex is a black comedy play directed by harry burton. it is based on the 2005 script by clive exton and made its world premiere at the wyndham's theatre in september 2013.
559996,13,Science & Spirit,science & spirit science & spirit is a discontinued american bimonthly magazine that covered scientific stories with an eye toward their spiritual implications. it was launched by the john templeton foundation in 1989 as a newsletter converted to a glossy magazine in 1998 then repositioned for a general readership in 2001. in 2003 it was purchased by heldref publications though the john templeton foundation continued to provide editorial support.
559997,13,The Blithedale Romance,the blithedale romance the blithedale romance (1852) is nathaniel hawthorne's third major romance. in hawthorne (1879) henry james called it the lightest the brightest the liveliest of hawthorne's unhumorous fictions.
559998,13,Razadarit Ayedawbon,razadarit ayedawbon razadarit ayedawbon (burmese: ရာဇာဓိရာဇ် အရေးတော်ပုံ) is a burmese chronicle covering the history of ramanya from 1287 to 1421. the chronicle consists of accounts of court intrigues rebellions diplomatic missions wars etc. about half of the chronicle is devoted to the reign of king razadarit (r.


In [29]:
data = {'labels': dbpedia_dataset['test']['labels'],
        'title': dbpedia_dataset['test']['title'],
        'content': dbpedia_dataset['test']['content']}

dbpedia_test_df = pd.DataFrame(data)
del data
dbpedia_test_df['content'] = dbpedia_test_df['content'].str.lower()
dbpedia_test_df

,labels,title,content
0,0,TY KU,ty ku ty ku /taɪkuː/ is an american alcoholic beverage company that specializes in sake and other spirits. the privately-held company was founded in 2004 and is headquartered in new york city new york. while based in new york ty ku's beverages are made in japan through a joint venture with two sake breweries. since 2011 ty ku's growth has extended its products into all 50 states.
1,0,Odd Lot Entertainment,odd lot entertainment oddlot entertainment founded in 2001 by longtime producers gigi pritzker and deborah del prete (the wedding planner) is a film production and financing company based in culver city california.oddlot produced the film version of orson scott card's sci-fi novel ender's game. a film version of this novel had been in the works in one form or another for more than a decade by the time of its release.
2,0,Henkel,henkel henkel ag & company kgaa operates worldwide with leading brands and technologies in three business areas: laundry & home care beauty care and adhesive technologies. henkel is the name behind some of america’s favorite brands.
3,0,GOAT Store,goat store the goat store (games of all type store) llc is one of the largest retro gaming online stores and an independent video game publishing label. additionally they are one of the primary sponsors for midwest gaming classic.
4,0,RagWing Aircraft Designs,ragwing aircraft designs ragwing aircraft designs (also called the ragwing aeroplane company and ragwing aviation) was an american aircraft design and manufacturing company based in belton south carolina.
...,...,...,...
69995,13,Energy Victory,energy victory energy victory: winning the war on terror by breaking free of oil is a 2007 book by robert zubrin. zubrin's central argument is that the decisive front in the war on terror is america's struggle for energy independence. he outlines the manner in which radical islam has been financed by oil revenues the technological feasibility of ethanol-fueled vehicles as well as the economic and agricultural imperatives for ethanol production and the environmental implications of his plan.
69996,13,Bestiario,bestiario bestiario is a book of 8 short stories written by julio cortázar.
69997,13,Wuthering Heights,wuthering heights wuthering heights is a novel by emily brontë written between october 1845 and june 1846 and published in 1847 under the pseudonym ellis bell. it was her first and only published novel: she died the following year at age 30. the decision to publish came after the success of her sister charlotte's novel jane eyre.
69998,13,L'Indépendant,l'indépendant l'indépendant is a newspaper published in luxembourg from 1945.


## A gentle introduction to LFs

Labeling functions (LFs) help users encode domain knowledge and other supervision sources programmatically.

LFs are heuristics that take as input a data point and either assign a label to it (in this case, HAM or SPAM) or abstain (don’t assign any label). Labeling functions can be noisy: they don’t have perfect accuracy and don’t have to label every data point. Moreover, different labeling functions can overlap (label the same data point) and even conflict (assign different labels to the same data point). This is expected, and we demonstrate how we deal with this later.

Because their only requirement is that they map a data point a label (or abstain), they can wrap a wide variety of forms of supervision. Examples include, but are not limited to:

    Keyword searches: looking for specific words in a sentence
    Pattern matching: looking for specific syntactical patterns
    Third-party models: using an pre-trained model (usually a model for a different task than the one at hand)
    Distant supervision: using external knowledge base
    Crowdworker labels: treating each crowdworker as a black-box function that assigns labels to subsets of the data

### a) Exploring the training set for initial ideas

We’ll start by looking at 20 random data points from the train set to generate some ideas for LFs.


In [30]:
Abstain = -1
Company = 0
EducationalInstitution = 1
Artist = 2
Athlete = 3
OfficeHolder = 4
MeanOfTransportation = 5
Building = 6
NaturalPlace = 7
Village = 8
Animal = 9
Plant = 10
Album = 11
Film = 12
WrittenWork = 13

In [31]:
pd.set_option('display.max_colwidth', None)

In [32]:
dbpedia_train_df[dbpedia_train_df['labels'] == 7].sample(50, random_state=42)

,labels,title,content
312823,7,Hedströmmen,hedströmmen hedströmmen is a river located in bergslagen in sweden.
296298,7,Gooseneck Glacier,gooseneck glacier gooseneck glacier is located in the fitzpatrick wilderness shoshone national forest in the us state of wyoming. the glacier is east of the continental divide in the northern wind river range and on the southeast flank of gannett peak the tallest mountain in wyoming. gooseneck glacier is separated from dinwoody glacier by a rocky outcropping and they are part of the largest grouping of glaciers in the american rocky mountains.
308505,7,Chiascio,chiascio the chiascio is a river of umbria central italy.it is part of the tiber basin.
286689,7,Lee (Vechte),lee (vechte) lee is a river of lower saxony germany.
306893,7,McClellan Creek,mcclellan creek mcclellan creek is a river in texas.named after george b. mcclellan who with his father-in-law randolph b. marcy made a survey of the area in 1851–52 looking for a route for the southern pacific railroad.
316572,7,Smoky Mountains (Idaho),smoky mountains (idaho) the smoky mountains are part of the rocky mountains and located on the west side of the wood river valley near sun valley idaho in the western united states. the range is within sawtooth national forest while part of it is within the sawtooth national recreation area.
292335,7,Maebongsan (Inje Gangwon-do),maebongsan (inje gangwon-do) maebongsan is a mountain in the county of inje gangwon-do in south korea. it has an elevation of 1271.1 m (4170 ft).
309591,7,Mount Colden,mount colden mount colden is the eleventh highest peak in the high peaks of the adirondack mountains new york usa.the peak is named after david s. colden an investor in the mcintyre iron works in 1836. the peak was briefly renamed mount mcmartin the next year but the older name persisted. the mountain is known for its distinctive trap dike a large crevice running up the center of the mountain which can clearly be seen from avalanche lake.there are two maintained trails up mount colden.
298948,7,Mount Angeles,mount angeles mount angeles is located just south of port angeles washington in the olympic national park. it is the highest peak in the hurricane ridge area. the summit which offers panoramic views of the strait of juan de fuca and many of the peaks of the interior olympic mountains can be climbed from the klahane ridge trail.the southeast side of mount angeles drains into morse creek thence into the strait of juan de fuca a couple of miles east of port angeles.
311067,7,Onatah Corona,onatah corona onatah corona is a feature on venus adjacent to ba'het corona. both features are surrounded by a ring of ridges and troughs which in places cut more radially-oriented fractures. the centers of the features also contain radial fractures as well as volcanic domes and flows. coronae are thought to form due to the upwelling of hot material from deep in the interior of venus.


Some things to notice for each label:

0. Company:

Words like: company, headquarter, corporate, finance, Ltd., airline, firm, commerce, manufacturer, factory, based in, based out of, founded, corporation, Inc., Foundation, newspaper, 

1. EducationalInstitution

Words like: University, students, bachelor, degree, school, academy, college, 

2. Artist

Words like: dancer, writer, artist, actor, singer-songwriter, teacher, scholar, professor, composer, actress, pianist, novelist, singer, songwriter, born, English, American, Chinese, guitarist, author, drummer, vocalist, saxophonist, painter, Canadian, member of, film, screenwriter, prose, poet, career, jazz, folk, 

3. Athlete

Words like: rugby, player, football, professional, NFL, League, injury, 1st round, contract, footballer, wrestling, lineman, cricketer, born, retired, former, MLB, pitcher, fencer, driver, American, Canadian, English, Belgian, attended, season, champion, motorcycle

4. OfficeHolder

Words like: born, author, commentator, director, professor, leader, Governor, politician, Minister, President, died, representatives, assembly, Republic, Democratic, House of Representatives, House, election, Member of Parliament, Party, Speaker, president, lawyer,  Liberal, candidate, election, Deputy, Prime Minister, official

5. MeanOfTransportation

Words like: 

6. Building
7. NaturalPlace
8. Village
9. Animal
10. Plant
11. Album
12. Film
13. WrittenWork

In [16]:
dbpedia_train_df['content'].str.lower()

0                                                                                                                                                                                                                                                     e. d. abbott ltd  abbott of farnham e d abbott limited was a british coachbuilding business based in farnham surrey trading under that name from 1929. a major part of their output was under sub-contract to motor vehicle manufacturers. their business closed in 1972.
1                                                                                                                                                                                                                                                                                                     schwan-stabilo  schwan-stabilo is a german maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. it is the world's largest manufacturer of highlig

In [ ]:
from snorkel.labeling import labeling_function

@labeling_function()
def 

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = []

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=dbpedia_train_df)

In [ ]:
L_train

In [ ]:
# We can easily calculate the coverage of these LFs 
# (i.e., the percentage of the dataset that they label) as follows:
coverage_check_out, coverage_check = (L_train != ABSTAIN).mean(axis=0)
print(f"check_out coverage: {coverage_check_out * 100:.1f}%")
print(f"check coverage: {coverage_check * 100:.1f}%")

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
dbpedia_train_df.iloc[L_train[:, 1] == Company].sample(10, random_state=1)

In [ ]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 0], L_train[:, 1])
dbpedia_train_df.iloc[buckets[(Abstain, Company)]].sample(10, random_state=1)